In [ ]:
import subprocess
import neutralb1.utils

WORKSPACE_DIR = neutralb1.utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

**Repository Version** 

This notebook was run at commit:
``

# Acceptance-Correcting Projected Moments
When partial wave fits are performed to data that includes detector acceptance effects, we typically perform an "acceptance correction" to remove those effects so that we obtain the *true physics* of the decay. If we want to project out moments from partial wave results, these partial waves must be acceptance corrected, otherwise the moments will be measured quantities that aren't of particular interest to us. In this study we'll verify that we are projecting and acceptance correcting properly by comparing moment values obtained from fits to **thrown** Monte Carlo (no detector acceptance) vs **accepted** Monte Carlo, were we've applied detector acceptance and other cuts to the thrown data.

By performing partial wave fits in both datasets, we can check whether the acceptance-corrected projected moments of the accepted dataset match the projected moments of the thrown dataset. We also have access to the fitted moments in the thrown case, which can provide a further check that the projection is working well. Unfortunately, the fitted moments currently have no method to be
acceptance-corrected, and so we can not fit moments to the accepted dataset.

This notebook follows naturally from the previous [moment verification study](./verify_moment.ipynb)

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import scipy.stats

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/input-output-tests/proj-acc-correct"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

Lets print out what parameters we've submitted the fit with. These are the same for thrown and accepted MC, except that `data_option` is changed to `'_mc'`. These fits are then run with `uv run submit submission.YAML`

In [ ]:
%%bash
cat $STUDY_DIR/submission.YAML

We then transfer only the necessary results of our fits to our study directory. First the thrown directory

In [ ]:
%%bash 
cd /lustre24/expphy/volatile/halld/home/kscheuer/ampToolsFits/omegapi/allPeriods/PARA_0/ver03.1_mcthrown/ver03/1m_1p_iso/recoil_pi_mass_0.0/t_0.10-0.20/mass_1.200-1.220/
cp best.csv "$STUDY_DIR/csv/thrown.csv"
cp data.csv "$STUDY_DIR/csv/thrown_data.csv"
cp best_projected_moments.csv "$STUDY_DIR/csv/thrown_projected_moments.csv"
cp distributions/angles.pdf "$STUDY_DIR/thrown_angles.pdf"

cp truth/best.csv "$STUDY_DIR/csv/thrown_truth.csv"
cp truth/best_projected_moments.csv "$STUDY_DIR/csv/thrown_truth_projected_moments.csv"

And now for the accepted MC

In [ ]:
%%bash
cd /lustre24/expphy/volatile/halld/home/kscheuer/ampToolsFits/omegapi/allPeriods/PARA_0/ver03.1_mc/ver03/1m_1p_iso/recoil_pi_mass_0.0/t_0.10-0.20/mass_1.200-1.220/
cp best.csv "$STUDY_DIR/csv/acceptance.csv"
cp best_corrected.csv "$STUDY_DIR/csv/acceptance_corrected.csv"
cp data.csv "$STUDY_DIR/csv/acceptance_data.csv"
cp bootstrap/bootstrap.csv "$STUDY_DIR/csv/acceptance_bootstrap.csv"
cp bootstrap/bootstrap_corrected.csv "$STUDY_DIR/csv/acceptance_corrected_bootstrap.csv"
cp bootstrap/bootstrap_projected_moments.csv "$STUDY_DIR/csv/acceptance_bootstrap_projected_moments.csv"

cp best_projected_moments.csv "$STUDY_DIR/csv/acceptance_projected_moments.csv"
cp distributions/angles.pdf "$STUDY_DIR/acceptance_angles.pdf"


cp truth/best.csv "$STUDY_DIR/csv/acceptance_truth.csv"
cp truth/best_corrected.csv "$STUDY_DIR/csv/acceptance_corrected_truth.csv"
cp truth/best_projected_moments.csv "$STUDY_DIR/csv/acceptance_truth_projected_moments.csv"

Lets now load in our data files into pandas dataframes

In [ ]:
thrown_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown.csv")
thrown_data_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_data.csv")
thrown_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_projected_moments.csv")

truth_thrown_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_truth.csv")
truth_thrown_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/thrown_truth_projected_moments.csv")


acc_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance.csv")
acc_corrected_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_corrected.csv")
acc_data_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_data.csv")
acc_bootstrap_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_bootstrap.csv")
acc_corrected_bootstrap_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_corrected_bootstrap.csv")
acc_bootstrap_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_bootstrap_projected_moments.csv")
acc_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_projected_moments.csv")

truth_acc_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_truth.csv")
truth_acc_corrected_waves_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_corrected_truth.csv")
truth_acc_corrected_moments_df = pd.read_csv(f"{STUDY_DIR}/csv/acceptance_truth_projected_moments.csv")

It'll be much easier to manage the thrown and accepted signal MC results with the result manager

In [ ]:
thrown_result = ResultManager(
    fit_df = thrown_waves_df,
    data_df = thrown_data_df,
    proj_moments_df=thrown_moments_df,    
    truth_df = truth_thrown_waves_df,
    truth_proj_moments_df=truth_thrown_moments_df,
)

# projected moments are always acceptance corrected currently, so they're not included in this result
acc_result = ResultManager(
    fit_df = acc_waves_df,
    data_df = acc_data_df,
    bootstrap_df = acc_bootstrap_df,
    truth_df = truth_acc_waves_df,
)

acc_corrected_result = ResultManager(
    fit_df = acc_corrected_waves_df,
    data_df = acc_data_df,
    bootstrap_df = acc_bootstrap_df,
    proj_moments_df = acc_moments_df,
    bootstrap_proj_moments_df = acc_bootstrap_moments_df,
    truth_df = truth_acc_corrected_waves_df,
    truth_proj_moments_df = truth_acc_corrected_moments_df,
)

thrown_result.preprocess()
acc_result.preprocess()
acc_corrected_result.preprocess()

## Analysis

### Fits to Distributions
Let's first make sure the angular distributions look alright

In [ ]:
utils.display_pdf(f"{STUDY_DIR}/thrown_angles.pdf", page=0, resolution=150)
utils.display_pdf(f"{STUDY_DIR}/acceptance_angles.pdf", page=0, resolution=150)

### Checking the Acceptance Correction va Truth Fits
We can see how the acceptance correction effects our results by comparing the truth fits in the thrown and accepted cases. A "truth" fit is one where the production coefficients are fixed to their generated values, and only a common scale factor is allowed to float so that the intensity can be adjusted to the number of events. This is because the generated production coefficients are sensitive to the intensity they were fit with, and so generated or accepted MC will have a different number of events. In either thrown or accepted cases, the production coefficients will be the same, just with a modified scale factor.

#### Partial Wave Fits
Let's first see how the waves were effected by comparing the ratio $A_{\text{acc-corrected}} / A_{\text{thrown}}$

In [ ]:
amplitudes = utils.get_coherent_sums(acc_corrected_result.truth_df)["eJPmL"]

ratios = []
for amp in amplitudes:
    acc_value = acc_corrected_result.truth_df[amp].values[0]
    thrown_value = thrown_result.truth_df[amp].values[0]    
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    ratios.append(ratio)

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(amplitudes, ratios)
ax.set_ylabel(r"$A_{\text{acc-corrected}} ~/~ A_{\text{thrown}}$")
ax.set_title("Ratio of Acceptance Corrected to Thrown Amplitudes")
ax.set_xticks(range(len(amplitudes)))
ax.set_xticklabels([utils.convert_amp_name(amp) for amp in amplitudes], rotation=45)
ax.set_ylim(0.9, 1.01)
plt.show()

There seems to be a common factor missing. We can notice that the number of events that we get when acceptance correcting does not match the number of events that we generated with

In [ ]:
print(f"Truth Thrown Generated Events: \t\t\t{thrown_result.fit_df['generated_events'].values[0]}")
print(f"Truth Acceptance Corrected Generated Events: \t{acc_corrected_result.fit_df['generated_events'].values[0]}")

percent_diff = 100 * (acc_corrected_result.fit_df['generated_events'].values[0] - thrown_result.fit_df['generated_events'].values[0]) / thrown_result.fit_df['generated_events'].values[0]
print(f"Percent Difference in Generated Events: \t{percent_diff:.2f}%")

If we divide out this value for each case, we can check if the ratios now deviate from 1

In [ ]:
amplitudes = utils.get_coherent_sums(acc_corrected_result.truth_df)["eJPmL"]

ratios = []
for amp in amplitudes:
    acc_value = acc_corrected_result.truth_df[amp].values[0] / acc_corrected_result.truth_df["generated_events"].values[0]
    thrown_value = thrown_result.truth_df[amp].values[0] / thrown_result.truth_df["generated_events"].values[0]
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    ratios.append(ratio)

if not np.allclose(ratios, 1.0, atol=1e-2):
    raise ValueError("Warning: Acceptance correction may not be working well, ratios deviate from 1 by more than 1%")    

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(amplitudes, ratios)
# \text{Intensity_{\text{acc-corrected}}}
ax.set_ylabel(r"$\dfrac{\left(\frac{A}{Intensity}\right)_{\text{acc-corrected}}}{\left(\frac{A}{Intensity}\right)_{\text{thrown}}} $")
ax.set_title("Ratio of Acceptance Corrected to Thrown Amplitude Fit Fractions")
ax.set_xticks(range(len(amplitudes)))
ax.set_xticklabels([utils.convert_amp_name(amp) for amp in amplitudes], rotation=45)
ax.set_ylim(0.8, 1.01)
plt.show()

Lastly, we can check the phase differences

In [ ]:
pds = list(utils.get_phase_differences(thrown_result.truth_df))

ratios = []
for pd in pds:
    acc_value = acc_corrected_result.truth_df[pd].values[0]
    thrown_value = thrown_result.truth_df[pd].values[0]    
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan    
    ratios.append(ratio)

fig, ax = plt.subplots(figsize=(25, 5))
ax.bar(pds, ratios)
ax.set_ylabel(r"$A_{\text{acc-corrected}} ~/~ A_{\text{thrown}}$")
ax.set_title("Ratio of Acceptance Corrected to Thrown Amplitudes")
ax.set_xticks(range(len(pds)))
ax.set_xticklabels([utils.convert_amp_name(pd) for pd in pds], rotation=45, ha='right', rotation_mode='anchor')
ax.set_ylim(0.85, 1.01)
plt.show()

There appear to be two problematic phase differences, but we can see below that their values are extremely small. Since these are truth fits, they include Breit-Wigners which modulate the phase difference values. We corrected for this in the preprocess method of our ResultManager class earlier, but for small phase differences it likely introduces some error thats unaccounted for

In [ ]:
small_pds = ["p1ppD_p1m0P", "m1pmS_m1m0P"]
for pd in small_pds:
    acc_value = acc_corrected_result.truth_df[pd].values[0]
    thrown_value = thrown_result.truth_df[pd].values[0]    
    print(f"{pd}: acc={acc_value:.6f}, thrown={thrown_value:.6f}, ratio={acc_value/thrown_value:.6f}")

#### Projected Moments
Since the amplitudes are fixed, we should expect to find the same behavior in the projected moments

In [ ]:
moment_columns = [col for col in thrown_result.truth_proj_moments_df.columns if col.startswith("H")]

truth_moment_ratios = []
for m in moment_columns:
    acc_value = acc_corrected_result.truth_proj_moments_df[m].values[0]
    thrown_value = thrown_result.truth_proj_moments_df[m].values[0]
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    truth_moment_ratios.append(ratio)

fig, ax = plt.subplots(figsize=(20, 5))
ax.bar(moment_columns, truth_moment_ratios)
ax.set_ylabel(r"$H_{\text{acc-corrected}} ~/~ H_{\text{thrown}}$")
ax.set_title("Ratio of Acceptance Corrected to Thrown Projected Moments")
ax.set_xticks(range(len(moment_columns)))
ax.set_xticklabels(moment_columns, rotation=90)
ax.set_ylim(0.9, 1.01)
plt.show()

Now perform the same division by the relative number of events

In [ ]:
truth_moment_ratios = []
for m in moment_columns:
    acc_value = acc_corrected_result.truth_proj_moments_df[m].values[0] / acc_corrected_result.truth_df["generated_events"].values[0]
    thrown_value = thrown_result.truth_proj_moments_df[m].values[0] / thrown_result.truth_df["generated_events"].values[0]
    ratio = acc_value / thrown_value if thrown_value != 0 else np.nan
    truth_moment_ratios.append(ratio)

if not np.allclose(truth_moment_ratios, 1.0, atol=1e-2):
    raise ValueError("Warning: Acceptance correction may not be working well, ratios deviate from 1 by more than 1%")

fig, ax = plt.subplots(figsize=(20, 5))
ax.bar(moment_columns, truth_moment_ratios)
ax.set_ylabel(r"$\dfrac{\left(\frac{H}{Intensity}\right)_{\text{acc-corrected}}}{\left(\frac{H}{Intensity}\right)_{\text{thrown}}} $", fontsize=18)
ax.set_title("Ratio of Acceptance Corrected to Thrown Projected Moment Fit Fractions")
ax.set_xticks(range(len(moment_columns)))
ax.set_xticklabels(moment_columns, rotation=90)
ax.set_ylim(0.9, 1.01)
plt.show()

### Evaluating the Fit Result to the Accepted Monte Carlo
With acceptable acceptance-correction from the previous section, we can turn our attention to the fit results in the accepted Monte Carlo. Here we have performed 100 randomized fit results, in which we will only look at the fit result with the best likelihood out of those 100. This "best fit" was used to seed 300 bootstrap fits so that we can obtain more reasonable errors on the partial wave results. The projected moments currently have no method for error propagation from the PWA results, so the bootstrap also provides us with projected moment errors we would otherwise lack.

#### Accepted Partial Wave Result
To understand how the result of our partial wave fit performed, we can compare it to the truth information. We'll do this first for the acceptance-included Monte Carlo, just to avoid any possible issues in the acceptance correction. We'll do the comparison by calculating the weighted residuals
$$
\frac{A_{\text{acc}} - A_{\text{truth-acc}}}{\sigma_{A,\text{acc}}}
$$
Lets first check that the bootstrap uncertainties are reasonable

In [ ]:
amplitude = utils.get_coherent_sums(acc_result.fit_df)["eJPmL"]

relative_uncertainties = []
fit_fractions = []
intensity = acc_result.fit_df.loc[0, "generated_events"]
for amp in amplitudes:
    value = acc_result.fit_df.loc[0, amp]
    err = acc_result.bootstrap_df[f"{amp}"].std()
    rel_unc = abs(err / value) if value != 0 else np.nan
    relative_uncertainties.append(rel_unc)
    fit_frac = value / intensity if intensity != 0 else np.nan
    fit_fractions.append(fit_frac)

fig, axs = plt.subplots(2, 1, figsize=(14, 5), sharex=True)

axs[0].bar(amplitudes, relative_uncertainties)
axs[0].set_ylabel(r"$\sigma_{|A|^2} ~/~|A|^2$")
axs[0].set_title("Relative Uncertainty for Each Amplitude")
axs[0].tick_params(axis='x', rotation=45)

axs[1].bar(amplitudes, fit_fractions, yerr=[
    acc_result.bootstrap_df[amp].std() / intensity if intensity != 0 else 0 for amp in amplitudes
])
axs[1].set_xticks(range(len(amplitudes)))
axs[1].set_xticklabels([utils.convert_amp_name(amp) for amp in amplitudes])
axs[1].set_ylabel(r"$|A|^2 /$ Intensity")
axs[1].set_title("Fit Fraction for Each Amplitude")
axs[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate relative uncertainties for phase differences using bootstrap
phase_diffs = list(utils.get_phase_differences(acc_result.fit_df))
relative_uncertainties_pd = []
avg_fit_fractions = []

intensity = acc_result.fit_df.loc[0, "generated_events"]
for pd in phase_diffs:
    val = acc_result.fit_df.loc[0, pd]    
    val_err = np.rad2deg(scipy.stats.circstd(np.deg2rad(acc_result.bootstrap_df[pd]), low=-np.pi, high=np.pi))
    rel_unc = abs(val_err / val) if val != 0 else np.nan
    relative_uncertainties_pd.append(rel_unc)

    # Get the two amplitudes involved in the phase difference
    amp1, amp2 = pd.split('_')
    fit_frac1 = acc_result.fit_df.loc[0, amp1] / intensity if intensity != 0 else np.nan
    fit_frac2 = acc_result.fit_df.loc[0, amp2] / intensity if intensity != 0 else np.nan
    avg_fit_fractions.append(0.5 * (abs(fit_frac1) + abs(fit_frac2)))

fig, axs = plt.subplots(2, 1, figsize=(18, 8), sharex=True)

axs[0].bar(phase_diffs, relative_uncertainties_pd)
axs[0].set_ylabel(r"$\sigma_{\eta_{A_1, A_2}} / \eta_{A_1, A_2}$", fontsize=20)
axs[0].set_title("Relative Uncertainty for Each Phase Difference (Accepted MC)")
axs[0].tick_params(axis='x', rotation=45)
axs[0].axhline(1, color='r', linestyle='-', linewidth=1)

axs[1].bar(phase_diffs, avg_fit_fractions)
axs[1].set_ylabel(r"$\frac{|A_1|^2 + |A_2|^2}{2 \times \text{Intensity}}$", fontsize=20)
axs[1].set_title("Average Fit Fraction of Amplitudes in Each Phase Difference")
axs[1].set_xticks(range(len(phase_diffs)))
axs[1].set_xticklabels([utils.convert_amp_name(pd) for pd in phase_diffs], rotation=45, ha="right", rotation_mode="anchor")
axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

plt.tight_layout()
plt.show()

Now we can calculate the weighted residuals for all amplitudes and phase differences

In [ ]:
# Calculate weighted residuals for each amplitude
weighted_residuals = []

for amp in amplitudes:     
    acc_val = acc_result.fit_df.loc[0, amp] 
    acc_err = acc_result.bootstrap_df[f"{amp}"].std()

    true_val = acc_result.truth_df.loc[0, amp]

    if acc_err != 0:
        residual = (acc_val - true_val) / acc_err
    else:
        residual = np.nan
    weighted_residuals.append(residual)

for pd in phase_diffs:
    acc_val = acc_result.fit_df.loc[0, pd]
    acc_err = np.rad2deg(scipy.stats.circstd(np.deg2rad(acc_result.bootstrap_df[pd]), low=-np.pi, high=np.pi))    
    true_val = acc_result.truth_df.loc[0, pd]
    # Use difference of absolute values due to phase ambiguity
    if acc_err != 0:
        residual = (np.abs(acc_val) - np.abs(true_val)) / acc_err
    else:
        residual = np.nan
    weighted_residuals.append(residual)

reduced_chi2 = np.nansum(np.array(weighted_residuals)**2) / (len(amplitudes) + len(phase_diffs) - 1)

plt.figure(figsize=(30, 5))
plt.bar(amplitudes + phase_diffs, weighted_residuals)
plt.ylabel(r"$\frac{A_{\text{acc}} - A_{\text{true}}}{\sigma_{A_{\text{acc}}}}$", fontsize=20)
plt.xticks(range(len(amplitudes + phase_diffs)), [utils.convert_amp_name(label) for label in amplitudes + phase_diffs], rotation=45)
plt.axhline(0, color='k', linestyle='--', linewidth=1)
plt.tight_layout()

plt.annotate(rf"$\chi^2 ~/~\text{{dof}} = {reduced_chi2:.2f}$", xy=(1, 1), xycoords='axes fraction',
            horizontalalignment='right', verticalalignment='top',
            fontsize=12)


plt.show()

#### Acceptance-Corrected Partial Wave Result
Before taking a look at the moments, we can check that our acceptance correction worked well for the partial waves first. We'll want to calculate the weighted residuals, or
$$
\frac{A_{\text{acc-corrected}}' - A_{\text{truth-thrown}}'}{\sigma_{A,\text{acc-corrected}}'}
$$
where the prime indicates values normalized by the relative number of generated events:
$$
A_{\text{acc-corrected}}' = \frac{A_{\text{acc-corrected}}}{\text{generated events}}
\,.
$$
But first we need to ensure the bootstrap uncertainties are reasonable

In [ ]:
amplitudes = utils.get_coherent_sums(acc_corrected_result.fit_df)["eJPmL"]

relative_uncertainties = []
fit_fractions = []
intensity = acc_corrected_result.fit_df.loc[0, "generated_events"]
for amp in amplitudes:
    value = acc_corrected_result.fit_df.loc[0, amp]
    err = acc_corrected_result.bootstrap_df[f"{amp}"].std()
    rel_unc = abs(err / value) if value != 0 else np.nan
    relative_uncertainties.append(rel_unc)
    fit_frac = value / intensity if intensity != 0 else np.nan
    fit_fractions.append(fit_frac)

fig, axs = plt.subplots(2, 1, figsize=(14, 5), sharex=True)

axs[0].bar(amplitudes, relative_uncertainties)
axs[0].set_ylabel(r"$\sigma_{|A|^2} ~/~|A|^2$")
axs[0].set_title("Relative Uncertainty for Each Amplitude (Acceptance Corrected)")
axs[0].tick_params(axis='x', rotation=45)

axs[1].bar(amplitudes, fit_fractions, yerr=[
    acc_corrected_result.bootstrap_df[amp].std() / intensity if intensity != 0 else 0 for amp in amplitudes
])
axs[1].set_xticks(range(len(amplitudes)))
axs[1].set_xticklabels([utils.convert_amp_name(amp) for amp in amplitudes])
axs[1].set_ylabel(r"$|A|^2 /$ Intensity")
axs[1].set_title("Fit Fraction for Each Amplitude (Acceptance Corrected)")
axs[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# TODO: find relative uncertainties for the phase differences

As expected, we have large uncertainties for the amplitudes with very small contributions to the intensity, which is okay. Now we can calculate the weighted residuals

In [ ]:
# Calculate weighted residuals for each amplitude
weighted_residuals = []
acc_intensity = acc_corrected_result.fit_df.loc[0, "generated_events"]
thrown_intensity = thrown_result.truth_df.loc[0, "generated_events"]

for amp in amplitudes:     
    acc_val = acc_corrected_result.fit_df.loc[0, amp] / acc_intensity
    acc_err = acc_corrected_result.bootstrap_df[f"{amp}"].std() / acc_intensity

    thrown_val = thrown_result.truth_df.loc[0, amp] / thrown_intensity

    if acc_err != 0:
        residual = (acc_val - thrown_val) / acc_err
    else:
        residual = np.nan
    weighted_residuals.append(residual)

plt.figure(figsize=(14, 4))
plt.bar(amplitudes, weighted_residuals)
plt.ylabel(r"$\frac{A'_{\text{acc}} - A'_{\text{thrown}}}{\sigma'_{A'_{\text{acc}}}}$", fontsize=20)
plt.xticks(range(len(amplitudes)), [utils.convert_amp_name(amp) for amp in amplitudes], rotation=45)
plt.axhline(0, color='k', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

In [ ]:
# TODO: take a look at non-acceptance corrected case to see if those largest amplitudes are also under-estimated

And do the same for the phase differences

In [ ]:
# TODO: weighted residuals of phase differences. Need to calculate absolute difference due to ambiguity

#### Projected Moments

In [ ]:
moments = [col for col in thrown_result.truth_proj_moments_df.columns if col.startswith("H")]

relative_uncertainties = []
fit_fractions = []
H0_0000 = acc_corrected_result.bootstrap_proj_moments_df.loc[0, "H0_0000"]
for mom in moments:
    value = acc_corrected_result.proj_moments_df.loc[0, mom]
    err = acc_corrected_result.bootstrap_proj_moments_df[mom].std()
    rel_unc = abs(err / value) if value != 0 else np.nan
    relative_uncertainties.append(rel_unc)
    fit_frac = value / H0_0000 if H0_0000 != 0 else np.nan
    fit_fractions.append(abs(fit_frac))

fig, axs = plt.subplots(2, 1, figsize=(14, 5), sharex=True)

axs[0].bar(moments, relative_uncertainties)
axs[0].set_ylabel(r"$\sigma_{H} ~/~ H$")
axs[0].set_title("Relative Uncertainty for Each Moment (Acceptance Corrected)")
axs[0].tick_params(axis='x', rotation=45)

axs[1].bar(moments, fit_fractions, yerr=[
    acc_corrected_result.bootstrap_proj_moments_df[mom].std() / H0_0000 if H0_0000 != 0 else 0 for mom in moments
])
axs[1].set_xticks(range(len(moments)))
axs[1].set_ylabel(r"$|H ~/~ H0\_0000|$ ")
axs[1].set_title("Normalized Moments (Acceptance Corrected)")
axs[1].tick_params(axis='x', rotation=90)
axs[1].set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
weighted_residuals_moments = []
acc_intensity = acc_corrected_result.fit_df.loc[0, "generated_events"]
thrown_intensity = thrown_result.truth_df.loc[0, "generated_events"]

for mom in moments:
    acc_val = acc_corrected_result.proj_moments_df.loc[0, mom] / acc_intensity
    acc_err = acc_corrected_result.bootstrap_proj_moments_df[mom].std() / acc_intensity
    thrown_val = thrown_result.truth_proj_moments_df.loc[0, mom] / thrown_intensity

    if acc_err != 0:
        residual = (acc_val - thrown_val) / acc_err #TODO: make the relative difference another plot
    else:
        residual = np.nan
    weighted_residuals_moments.append(residual)

reduced_chi2 = np.nansum(np.array(weighted_residuals_moments)**2) / (len(moments) - 1)
print(f"Reduced Chi^2 for Moments: {reduced_chi2:.2f}")

plt.figure(figsize=(16, 4))
plt.bar(moments, weighted_residuals_moments)
plt.ylabel(r"$\frac{H'_{\text{acc}} - H'_{\text{thrown}}}{\sigma'_{H'_{\text{acc}}}}$", fontsize=20)
plt.xticks(rotation=90)
plt.axhline(0, color='k', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate weighted residuals between thrown_fitted_moments_df and filtered_thrown_moments_df
residuals_truth_vs_fit_moments = []
for mom in moments:
    corrected_val = filtered_acc_corrected_moments_df.loc[0, mom]
    projected_val = filtered_truth_thrown_moments_df.loc[0, mom]
    # residuals_truth_vs_fit_moments.append((projected_val - corrected_val) / projected_val)
    residuals_truth_vs_fit_moments.append((projected_val / corrected_val))

plt.figure(figsize=(16, 4))
plt.bar(moments, residuals_truth_vs_fit_moments)
plt.ylabel("x")
plt.xticks(rotation=90)
plt.axhline(1, color='k', linestyle='--', linewidth=1)
plt.ylim(0,3)
plt.tight_layout()
plt.show()